In [20]:
from google.colab import drive
drive.mount('/content/drive')

ModuleNotFoundError: No module named 'google.colab'

In [ ]:
#importing os to access files from the system

In [ ]:
import os
import cv2
import imutils

In [ ]:
features=[]
target=[]

In [ ]:
for x in["clear face","dark spots","puffy eyes","wrinkles"]: #considering 4 possibilities clear face, dark spots, puffy eyes and wrinkles
    ImagesNamesList=os.listdir("/content/drive/MyDrive/verzeo_ageing_dataset (1)" + "/" + str(x))
    for y in ImagesNamesList:
        Imgarr=cv2.imread("/content/drive/MyDrive/verzeo_ageing_dataset (1)" + "/" + str(x) + "/" + y)
        try:
            Imgarr=cv2.resize(Imgarr,(244,244))
            features.append(Imgarr)
        except:
            pass
        else:
            if x=="clear face ":
                target.append(0)
            elif x=="dark spots":
                target.append(1) 
            elif x=="puffy eyes":
                target.append(2)
            else:
                target.append(3)
    print("In Folder", x)

In [ ]:
import numpy as np

In [ ]:
features=np.array(features)
target=np.array(target)

In [ ]:
features.shape

In [ ]:
target.shape

In [ ]:
#splitting the data into testing and training

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
features_train,features_test,target_train,target_test=train_test_split(features,target,test_size=0.2)

In [ ]:
features_train.shape

In [ ]:
target_train.shape

In [ ]:
features_test.shape

In [ ]:
target_test.shape

In [ ]:
#converting colored images to b/w

In [ ]:
def preprocessing(image):
    image=cv2.cvtColor(image,cv2.COLOR_BGR2GRAY)
    image=image/255 
    return image

In [ ]:
features_train=np.array(list(map(preprocessing,features_train)))

In [ ]:
features_train.shape

In [ ]:
features_train=features_train.reshape((1276, 244, 244,1))

In [ ]:
from keras.preprocessing.image import ImageDataGenerator

In [ ]:
dataGen=ImageDataGenerator(rotation_range=10,width_shift_range=0.1,height_shift_range=0.1,zoom_range=0.2,shear_range=0.1)

In [ ]:
dataGen.fit(features_train)

In [ ]:
batches=dataGen.flow(features_train,target_train,batch_size=20)

In [ ]:
images,labels=next(batches)

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
plt.figure(figsize=(10,10))
for i in range(0,20):
    plt.subplot(4,5,i+1)
    plt.imshow(images[i].reshape(244,244))
plt.show()

In [ ]:
from tensorflow.keras.utils import to_categorical

In [ ]:
target_train=to_categorical(target_train)

In [ ]:
target_train.shape

In [ ]:
features_train.shape

In [ ]:
# STEP 1: specifying the architecture

In [ ]:
from keras.layers import Dense,Flatten,Conv2D,MaxPooling2D,Dropout
from keras.models import Sequential
from tensorflow.keras.applications import EfficientNetB0


In [ ]:
conv_base=EfficientNetB0(weights="imagenet",include_top=False,input_shape=(244,244,3))
model=Sequential()
model.add(conv_base)
model.add(MaxPooling2D((3,3)))
model.add(Dropout(0.5))
model.add(Flatten())
model.add(Dense(4,activation="sigmoid"))

In [ ]:
from keras.optimizers import Adam

In [ ]:
# STEP 2: compiling the model

In [ ]:
model.compile(Adam(learning_rate=0.001),loss="binary_crossentropy",metrics=["accuracy"])

In [ ]:
# STEP 3: training the model

In [ ]:
model.fit(dataGen.flow(features_train,target_train,batch_size=20),epochs=20)

In [ ]:
# STEP 4: testing the model by localizing the ageing signs

In [ ]:
import numpy as np
import cv2
 

In [ ]:
from google.colab.patches import cv2_imshow
import argparse

image=cv2.imread("/content/drive/MyDrive/03198defbf9905613bedbbdd2323228d.jpg")
imagearr=np.asarray(image)
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades +"haarcascade_frontalface_default.xml")
gray=cv2.cvtColor(image,cv2.COLOR_BGR2GRAY)
faces=face_cascade.detectMultiScale(gray,1.05,5)
labels=np.array(['clear_face','wrinkles','puffy','dark spots'],dtype='object')
for (x,y,w,h) in faces:
    detectedFace=image[x:x+w,y:y+h]
    detectedFace=cv2.resize(detectedFace,(244,244))
    detectedFace=preprocessing(detectedFace)
    detectedFace=detectedFace.reshape(1,244,244,1)
    predictions=model.predict(detectedFace)[0]
     
    cv2.rectangle(image,(x,y),(x+w,y+h),(0,0,255),2)
    count =1
    for i in predictions:
        classname=labels[np.where(predictions==i)]
        accuracy=i*100
        label="{}:{:.2f}%".format(classname,accuracy)
        cv2.putText(image,label,(10,count *30+25),cv2.FONT_HERSHEY_SIMPLEX,1,(0,0,200),2)
        count +=1
        print(predictions)
    cv2_imshow(image)
    returnedValue=cv2.waitKey(1)
    if returnedValue==ord("s") or returnedValue==ord("S"):
        cv2.destroyAllWindows()
        break